https://dipbigdata.kr/pages/sub.htm?mode=view&mv_data=aWR4PTM5JnN0YXJ0UGFnZT0xNiZsaXN0Tm89NyZ0YWJsZT1jc19vbmxpbmVfZGF0YSZzaXRlX2NvZGU9ZGlwJm5hdl9jb2RlPWRpcDE2NjY3NzE5OTYmY29kZT0mc2VhcmNoX2l0ZW09JnNlYXJjaF9vcmRlcj0mb3JkZXJfbGlzdD0mbGlzdF9zY2FsZT0mdmlld19sZXZlbD0mdmlld19jYXRlPeu5heuNsOydtO2EsOu2hOyEneq4sOyCrCZ2aWV3X2NhdGUyPQ==||

작업유형2

1. 데이터 수집

- seaborn 라이브러리에서 제공하는 샘플 데이터를 활용하여 실습
- sns.get_dataset_names()로 샘플 데이터 종류를 확인할 수 있고 'penguins'활용
- 데이터를 로드하여 df 변수에 할당
- 종속 변수는 'species'열

In [2]:
import seaborn as sns
sns.get_dataset_names()
df = sns.load_dataset('penguins')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


2. 데이터 결측치 보정

- 연속형 변수 중 결측치가 존재하는 열을 확인 후 missing 리스트에 열이름을 입력
- for문으로 각 열별 중앙값을 결측치에 입력
- 성별과 같은 범주형의 경우 비중이 높은 남성으로 결측치 입력

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
missing = ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']

for i in missing:
    # 각 열별 중앙값을 결측치에 입력
    df[i] = df[i].fillna(df[i].median())

# 성별 결측치 남성으로
df['sex'] = df['sex'].fillna('Male')

3. 라벨인코딩

- 특정 변수의 경우 문자형으로 되어 있어 컴퓨터가 학습하기 위해 숫자형으로 변환이 필요
- sklearn라이브러리인 LabelEncoder모듈을 불러와 변환 작업 수행
- apply함수를 적용하여 label 변수에 리스트화 되어 있는 변수들의 라벨 인코딩 작업을 수행

In [6]:
# 문자형(object)
label = ['species','island','sex']

from sklearn.preprocessing import LabelEncoder
df[label] = df[label].apply(LabelEncoder().fit_transform)

In [8]:
for i in label:
    print(i,df[i].unique())

species [0 1 2]
island [2 0 1]
sex [1 0]


4. 데이터타입, 더미 변환

- pandas 라이브러리 불러오기
- 데이터타입을 카테고리형으로 변경할 열이름을 리스트로 만들기
- for문을 돌려서 카테고리 리스트의 열의 타입을 변경
- pd.get_dummies로 더미변수로 변환

In [9]:
import pandas as pd

category = ['island','sex']
for i in category:
    df[i] = df[i].astype('category')
df = pd.get_dummies(df)

In [10]:
df

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1
0,0,39.10,18.7,181.0,3750.0,False,False,True,False,True
1,0,39.50,17.4,186.0,3800.0,False,False,True,True,False
2,0,40.30,18.0,195.0,3250.0,False,False,True,True,False
3,0,44.45,17.3,197.0,4050.0,False,False,True,False,True
4,0,36.70,19.3,193.0,3450.0,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...
339,2,44.45,17.3,197.0,4050.0,True,False,False,False,True
340,2,46.80,14.3,215.0,4850.0,True,False,False,True,False
341,2,50.40,15.7,222.0,5750.0,True,False,False,False,True
342,2,45.20,14.8,212.0,5200.0,True,False,False,True,False


5. 파생변수 생성

- pandas의 qcut를 활용하여 특정변수의 분위수를 계산하여 이를 기반으로 이산화 수행
- 연속된 숫자값이 5개의 구간으로 범주화되어 파생변수가 생성

In [12]:
df['body_mass_g_qcut'] = pd.qcut(df['body_mass_g'],5,labels=False)
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1,body_mass_g_qcut
0,0,39.10,18.7,181.0,3750.0,False,False,True,False,True,1
1,0,39.50,17.4,186.0,3800.0,False,False,True,True,False,1
2,0,40.30,18.0,195.0,3250.0,False,False,True,True,False,0
3,0,44.45,17.3,197.0,4050.0,False,False,True,False,True,2
4,0,36.70,19.3,193.0,3450.0,False,False,True,True,False,0


6. 정규화 또는 스케일 작업

- 변수별 단위가 다른 부분에 대해서는 정규화 또는 스케일 작업이 필수
- 스케일할 변수를 리스트화
- sklearn의 MinMaxScaler 모듈을 활용하여 스케일 작업 수행
- fit한 다음 transform으로 데이터의 스케일 변환 진행

In [13]:
df.columns

Index(['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g', 'island_0', 'island_1', 'island_2', 'sex_0', 'sex_1',
       'body_mass_g_qcut'],
      dtype='object')

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm','body_mass_g']
m = MinMaxScaler()
m.fit(df[scaler])
df[scaler] = m.transform(df[scaler])

In [15]:
df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_0,island_1,island_2,sex_0,sex_1,body_mass_g_qcut
0,0,0.254545,0.666667,0.152542,0.291667,False,False,True,False,True,1
1,0,0.269091,0.511905,0.237288,0.305556,False,False,True,True,False,1
2,0,0.298182,0.583333,0.389831,0.152778,False,False,True,True,False,0
3,0,0.449091,0.500000,0.423729,0.375000,False,False,True,False,True,2
4,0,0.167273,0.738095,0.355932,0.208333,False,False,True,True,False,0


7. 데이터 분리

- sklearn model_selection의 train_test_split 로드
- 독립변수, 종속변수, 훈련용, 테스트 데이터 4개로 분리, 테스트 데이터의 비율은 0.2
- 종속변수를 기준으로 데이터의 비율을 감안하여 데이터를 분리
- random_state는 1

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df['species'],test_size=0.1,random_state=1,stratify=df['species'])

8. 모형 학습 (1)

- sklearn ensemble의 RandomForestClassifier 로드
- 랜덤 포레스트 분류모형 학습
- 모델 핏
- 테스트 데이터를 활용하여 예측한 결과값을 pred1에 할당

In [17]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier()
model1.fit(X_train,y_train)
pred1 = model1.predict(X_test)

9. 모형 학습 (2)

- sklearn ensemble의 AdaBoostClassifier
- 에이다부스트 분류모형 학습
- 모델 핏
- 테스트 데이터를 활용하여 예측한 결과값을 pred2에 할당

In [18]:
from sklearn.ensemble import AdaBoostClassifier
model2 = AdaBoostClassifier()
model2.fit(X_train,y_train)
pred2 = model2.predict(X_test)

10. 앙상블

- 앞서 구축된 2개 모형을 기반으로 보팅을 진행할 예정
- 보팅은 hard 투표기반으로 진행하며 옵션을 포함하여 clf에 할당
- 훈련용 데이터를 입력하여 clf 핏
- 테스트 데이터를 입력하여 예측한 결과값을 pred3에 할당

In [19]:
from sklearn.ensemble import VotingClassifier
clf = VotingClassifier(estimators=[('rf',model1),('ad',model2)],voting='hard')
clf.fit(X_train,y_train)
pred3 = clf.predict(X_test)

11. 모델 평가

- sklearn metrics의 accuracy_score 지표 활용
- classification_report의 경우 종합적인 지표를 알려주는 기능으로 참고
- 랜덤포레스트, 에이다, 보팅 3개 모형의 예측값과 실제값을 비교하여 print함수로 출력

In [20]:
from sklearn.metrics import accuracy_score

print('랜포 정확도',accuracy_score(y_test,pred1))
print('아이다 정확도',accuracy_score(y_test,pred2))
print('보팅 정확도',accuracy_score(y_test,pred3))

랜포 정확도 1.0
아이다 정확도 0.9714285714285714
보팅 정확도 1.0


12. 하이퍼파라미터 튜닝

- 해당부분은 실제 시험에서 필수사항은 아님
- 하지만 시험범위에서 하이퍼파라미터 튜닝을 포함하고 있기 때문에 해당 부분을 간단히 실습으로 참고
- sklearn model_selection의 GridSearchCV 로드
- 그리드 서치를 위한 파라미터 범위를 지정하고 parameters 변수에 할당
- 기본적인 랜덤포레스 모형 만들고, 그리드서치를 통해 최적의 파라미터 추정
- print함수를 통해 학습된 모형의 최적의 하이퍼파라미터를 best_praams_함수로 출력
- 여기에서 나온 값을 앞에 모형에 적용하여 최적화

In [21]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[50,100],
              'max_depth':[4,6]}
model4 = RandomForestClassifier()
clf = GridSearchCV(estimator=model4,param_grid=parameters,cv=3)
clf.fit(X_train,y_train)
print('최적',clf.best_params_)

최적 {'max_depth': 6, 'n_estimators': 100}


13. 저장 및 답안 출력

- y_test의 index를 id로 pred3을 pred열로 하는 데이터프레임을 만들고 이를 '003000000.csv'저장
- 여기서 index=True로 하게 되면 인덱스가 추가로 생성되므로 제출양식과 상이
- 파일 저장 후 다시 로드하여 제출파일을 확인

In [ ]:
pd.DataFrame({'id':y_test.index,'pred':pred3}).to_csv('003000000.csv',index=False)